<a href="https://colab.research.google.com/github/jihyoung-lee/stock_news_classifier/blob/main/stock_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [3]:
!pip install --upgrade transformers

In [3]:
!pip cache purge

Files removed: 30


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# model 과 tokenizer 불러오기
model_name = "klue/bert-base"
tokenization = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    ignore_mismatched_sizes=True,
    )

# data-set 로드(csv)
dataset = load_dataset('csv', data_files='/content/stock_news_labeled_auto.csv')

# 전처리 함수
def preprocess_function(examples):
    return tokenization(examples["title"], padding='max_length', truncation=True)

# 전처리
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# 훈련설정
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",
)

# 훈련
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    #eval_dataset=tokenized_dataset['test'], validation 나중에 같이 추가
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=39, training_loss=0.564268552339994, metrics={'train_runtime': 36.7861, 'train_samples_per_second': 8.237, 'train_steps_per_second': 1.06, 'total_flos': 79722649774080.0, 'train_loss': 0.564268552339994, 'epoch': 3.0})

In [13]:
trainer.save_model("./my_model")
tokenization.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json')

In [14]:
# 예측하기
model_path = "./my_model"
model = BertForSequenceClassification.from_pretrained(model_path, local_files_only=True)
tokenizer = BertTokenizer.from_pretrained(model_path, local_files_only=True)

text = "코스피 급락, 외국인 대량 매도"

# 입력 전처리
input = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

# 예측
output = model(**input)
logits = output.logits

#확률로 변환
import torch
probs = torch.softmax(logits, dim=1)

# 결과 출력
print(f"호재 확률: {probs[0][1].item():.4f}")
print(f"악재 확률: {probs[0][0].item():.4f}")

호재 확률: 0.4561
악재 확률: 0.5439
